# LAV BEV Training on Colab

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/MichaelBosello/carla-colab/blob/master/carla-simulator.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
</table>

---
---

## Introduction
[Carla](https://github.com/carla-simulator/carla) is an open-source simulator for autonomous driving research.

Since it is hardware demanding, it could be useful for students to run it on colab.

---

## Acknowledgements

Thanks to *Tomohiro*  for developing the code that allows remote access to colab.
Check out his [page](https://internet-of-tomohiro.netlify.app/google_colab/vnc.en.html) and [GitHub](https://github.com/demotomohiro/remocolab).



## Requirements

*   Ssh client (already present in all recent os).


## Installation 

4. Wait the end of the process
  * It sets up OpenSSH server, VirtualGL and TurboVNC.
  * Then, access passwords are shown



## Download Jekyll's LAV

Clone myown LAV from(https://github.com/jekyll88/lav)


In [1]:
! git clone https://github.com/jekyll88/lav.git


Cloning into 'lav'...
remote: Enumerating objects: 1961, done.
remote: Counting objects: 100% (1961/1961), done.
remote: Compressing objects: 100% (1827/1827), done.
remote: Total 1961 (delta 125), reused 1956 (delta 123), pack-reused 0
Receiving objects: 100% (1961/1961), 6.52 MiB | 8.59 MiB/s, done.
Resolving deltas: 100% (125/125), done.


## Install necessary packages

In [4]:
#! sudo -u colab mkdir /home/colab/carla
#! sudo -u colab tar -xf /home/colab/CARLA.tar.gz -C /home/colab/carla
!wget https://carla-releases.s3.eu-west-3.amazonaws.com/Linux/CARLA_0.9.13.tar.gz
!tar xvzf CARLA_0.9.13.tar.gz
!pip3 install einops
import sys
sys.path.append('/content/lav')

Streaming output truncated to the last 5000 lines.
Engine/Content/Slate/Common/Window/WindowButton_Close_Hovered.png
Engine/Content/Slate/Common/Window/WindowButton_Minimize_Pressed.png
Engine/Content/Slate/Common/Window/WindowTitle.png
Engine/Content/Slate/Common/Window/WindowButton_Minimize_Disabled.png
Engine/Content/Slate/Common/Window/WindowButton_Minimize_Normal.png
Engine/Content/Slate/Common/Window/WindowBackground.png
Engine/Content/Slate/Common/Window/WindowBorder.png
Engine/Content/Slate/Common/Window/WindowTitle_Flashing.png
Engine/Content/Slate/Common/Window/WindowButton_Maximize_Hovered.png
Engine/Content/Slate/Common/Window/WindowButton_Close_Pressed.png
Engine/Content/Slate/Common/Window/WindowTitle_Inactive.png
Engine/Content/Slate/Common/Window/WindowButton_Restore_Normal.png
Engine/Content/Slate/Common/Window/WindowButton_Restore_Hovered.png
Engine/Content/Slate/Common/ColorSpectrum.png
Engine/Content/Slate/Common/ProgressBar_Fill.png
Engine/Content/Slate/Common/Tabl

## Preparing Data


In [3]:
import tqdm
import torch
from lav.lav_privileged import LAV
from lav.utils.datasets import get_data_loader
from lav.utils.logger import Logger


ModuleNotFoundError: ignored

## Training

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

# Import some useful packages for this homework


# !nvidia-smi # list your current GPU
